<h3>Librerías, paquetes y funciones usadas en el cuaderno</h3>

In [1]:
import cv2
import numpy as np

<h3>Función para la obtención del color del parqué y de la zona</h3>

In [2]:
# Función para manejar los clics del ratón
def get_color(event, x, y, flags, param):
    global selected_colors
    if event == cv2.EVENT_LBUTTONDOWN:
        selected_color = frame[y, x]  # Obtener el color en el punto clicado
        selected_colors.append(selected_color)
        print(f"Color seleccionado (BGR): {selected_color}")
        if len(selected_colors) == 2:  # Si ya se seleccionaron dos colores, cerrar la ventana
            cv2.destroyWindow("Selecciona un punto del parqué y un punto de la zona.")

<h3>Selección de colores del parqué y de la zona para generar una máscara</h3>

In [ ]:
# Leer el video
video_path = "./assets/videos/okc-nyk.mp4"
cap = cv2.VideoCapture(video_path)

# Variables globales para almacenar los colores seleccionados
selected_colors = []

ret, frame = cap.read()  # Leer el primer fotograma
if not ret:
    print("Error al leer el video.")
    cap.release()
    exit()

# Mostrar el primer fotograma para seleccionar colores
cv2.imshow("Selecciona un punto del parqué y un punto de la zona.", frame)
cv2.setMouseCallback("Selecciona un punto del parqué y un punto de la zona.", get_color)

print("1. Haz clic en el parquet para seleccionar un color.")
print("2. Haz clic en la zona bajo la canasta para seleccionar otro color.")


# Esperar hasta que el usuario seleccione dos colores
while len(selected_colors) < 2:
    cv2.waitKey(1)

# Convertir los colores seleccionados a espacio HSV
selected_colors_hsv = [cv2.cvtColor(np.uint8([[color]]), cv2.COLOR_BGR2HSV)[0][0] for color in selected_colors]

# Crear máscaras para cada color seleccionado
masks = []
for color_hsv in selected_colors_hsv:
    lower_bound_hsv = np.clip(color_hsv - [10, 50, 50], 0, 255)
    upper_bound_hsv = np.clip(color_hsv + [10, 50, 50], 0, 255)

    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(frame_hsv, lower_bound_hsv, upper_bound_hsv)
    masks.append(mask)

# Combinar las máscaras
combined_mask = cv2.bitwise_or(masks[0], masks[1])

# Invertir la máscara combinada y aplicar para eliminar las áreas seleccionadas
mask_inv = cv2.bitwise_not(combined_mask)
masked_frame = cv2.bitwise_and(frame, frame, mask=mask_inv)

# Mostrar los resultados
cv2.imshow("Original", frame)
cv2.imshow("Parque y zona eliminadas", masked_frame)

cv2.waitKey(0)
cv2.destroyAllWindows()
cap.release()


In [ ]:
import numpy as np
import cv2
from skimage import transform

# Captura cuatro clics del ratón (parte uno)
def get_points(event, x, y, flags, param):
    puntos = param["points"]
    img = param["image"]

    if event == cv2.EVENT_LBUTTONDOWN:  # Botón izquierdo
        puntos.append((x, y))  
        
        # Dibujar en la imagen
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], img)

        # Cuarto puntos, condición de parada
        if len(puntos) == 4:
            cv2.destroyAllWindows()
            print("Coordenadas imagen:", puntos)

# Movimiento del puntero (parte 2)
def mouse_event(event, x, y, flags, param):
    homo = param["tform"]
    img = param["image"]

    if event == cv2.EVENT_MOUSEMOVE:  
        # Transformar posición del ratón a imagen destino
        print("uwu")
        punto_A = np.array([[x, y]])  # Puntero 
        punto_B = homo(punto_A)  # Transformar a imagen destino

        # Dibuja en la imagen destino
        print("wuwu")
        imgtmp = img.copy()
        cv2.circle(imgtmp, (int(punto_B[0][0]), int(punto_B[0][1])), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], imgtmp)

############## 1. RECOPILA LOS CUATRO PUNTOS EN AMBAS IMÁGENES
#Inicializa listas depuntos
puntosA = []
puntosB = []

video_path = "./assets/videos/okc-nyk.mp4"
cap = cv2.VideoCapture(video_path)

# Variables globales para almacenar los colores seleccionados
selected_colors = []

ret, frame = cap.read()  # Leer el primer fotograma
if not ret:
    print("Error al leer el video.")
    cap.release()
    exit()

#Lee imágenes, crea copias de trabajo
diagrama = cv2.imread('./assets/court_diagrams/black_court.png')

frametmp = frame.copy()
diagramatmp = diagrama.copy()

#Vista del campo
cv2.imshow("Vista", frametmp) 
params = {
    "points": puntosA,
    "image": frametmp, 
    "wname": "Vista"
}
cv2.setMouseCallback("Vista", get_points, params)

# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)

#Vista ceintal
cv2.imshow("Mapa", diagrama) 
params = {
    "points": puntosB,
    "image": diagramatmp, 
    "wname": "Mapa"
}
cv2.setMouseCallback("Mapa", get_points, params)

# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)
cv2.destroyAllWindows()

print("hola")
print("Coordenadas campo:", puntosA)
print("Coordenadas diagrama:", puntosB)
#Transformación de los puntos https://scikit-image.org/docs/stable/auto_examples/transform/plot_transform_types.html
tform = transform.estimate_transform('projective', np.array(puntosA), np.array(puntosB))
print("adios")

###### 2. MUESTRA POSICIÓN DEL RATÓN EN EL CAMPO, EN LA IMAGEN DEL MAPA
# Mostrar las dos imágenes
cv2.imshow("Vista", frame)
cv2.imshow("Mapa", diagrama)

params = {
    "tform": tform,
    "image": diagrama, 
    "wname": "Mapa"
}

# Evento de movimiento del ratón
cv2.setMouseCallback("Vista", mouse_event, params)

cv2.waitKey(0)
cv2.destroyAllWindows()